In [281]:
import pandas as pd
import numpy as np

In [282]:
with open("datass/Files/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    word_to_vec_map = {}
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_to_vec_map[word] = vector

In [283]:
train_data = pd.read_csv('datass/Files/train_emoji.csv')
test_data = pd.read_csv('datass/Files/tesss.csv')

In [284]:
X_test = test_data['I want to eat\t'].map(lambda x:x.replace("\t",""))
y_test = test_data['4']

In [285]:
for i in range(5):
    print(f" The sentence : {X_test[i]} , The emoji : {label_to_emoji[y_test[i]]}") 

 The sentence : he did not answer , The emoji : 😔
 The sentence : he got a very nice raise , The emoji : 😀
 The sentence : she got me a nice present , The emoji : 😀
 The sentence : ha ha ha it was so funny , The emoji : 😀
 The sentence : he is a good friend , The emoji : 😀


In [286]:
print(y_test.shape)
print(X_test.shape)

(55,)
(55,)


In [287]:
X_train = train_data['never talk to me again']
y_train = train_data['3']
X_train[131] = 'never talk to me again'
y_train[131] = 3

/tmp/ipykernel_100980/3497168708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[131] = 'never talk to me again'
/tmp/ipykernel_100980/3497168708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train[131] = 3


In [288]:
print(X_train.shape)
y_train.shape

(132,)


(132,)

In [289]:
label_to_emoji = {0:"❤️",1:"⚾",2:"😀",3:"😔",4:"🍴"}

In [291]:
# BASIC EDA
# What the data looks like
for i in range(5):
    print(X_train[i])
    print(y_train[i])

I am proud of your achievements
2
It is the worst day in my life
3
Miss you so much
0
food is life
4
I love you mum
0


In [292]:
# Emoji-Sentence pairs
for i in range(5):
    print(f" The sentence : {X_train[i]} , The emoji : {label_to_emoji[y_train[i]]}") 

 The sentence : I am proud of your achievements , The emoji : 😀
 The sentence : It is the worst day in my life , The emoji : 😔
 The sentence : Miss you so much , The emoji : ❤️
 The sentence : food is life , The emoji : 🍴
 The sentence : I love you mum , The emoji : ❤️


In [293]:
# Number of unique values 
pd.unique(y_train)

array([2, 3, 0, 4, 1])

In [294]:
# Distribution
class_dist = [0]*5
for i in y_train:
    class_dist[i]+=1
class_dist

[22, 19, 38, 36, 17]

In [295]:
# Max length of a sentence
maxLen = 0
for i in range(len(X_train)):
    maxLen = max(maxLen,len(X_train[i].split()))
maxLen

10

In [296]:
# Sentence to average
# Every sentence is split up into words , embeddings of the words are generated with glove6b_50 and averaged
def sentence_to_emb(sentence):
    X = sentence.lower().split()
    X = [word_to_vec_map[x] for x in X]
    avg = np.zeros((50,))
    for x in X:
        avg+=x
    avg = avg/len(sentence.split())
    return avg
sentence_to_emb(X_train[0])

array([ 0.00563117,  0.59711333, -0.13939716, -0.38432499,  0.74846166,
       -0.34453833, -0.09402834,  0.13647616, -0.27890999,  0.07412743,
       -0.05458783,  0.26998334, -0.40014283, -0.29957134,  0.50517549,
        0.13981333,  0.06849916,  0.53107068,  0.07820434, -0.29929733,
       -0.27029932,  0.49722101,  0.24299683,  0.03004166,  0.92919667,
       -1.64110001, -1.13771501,  0.00513   ,  0.45910502, -0.29221168,
        2.87481666,  0.687305  , -0.307625  , -0.16341315, -0.28182249,
       -0.1250206 ,  0.139661  ,  0.34585901, -0.04596499, -0.33912099,
       -0.02768405, -0.36635967, -0.164233  ,  0.32527532, -0.164843  ,
        0.216988  , -0.32414001, -0.06701233, -0.025773  ,  0.32180167])

In [297]:
def convert_to_one_hot(Y):
    one_hotted = []
    for i in Y:
        temp = [0]*5
        temp[i]+=1
        one_hotted.append(temp)
    return one_hotted
Y_oh = convert_to_one_hot(y_train)
Y_oh_test = convert_to_one_hot(y_test)

In [298]:
def model(X_train, y_train, learning_rate=0.01, epochs=500):
    W = np.random.randn(5, 50) / np.sqrt(50)
    b = np.zeros((5,))
    for epoch in range(1, epochs):
        cost = 0
        for i in range(len(X_train)):
            avg = sentence_to_emb(X_train[i])
            Z = np.dot(W, avg) + b
            expZ = np.exp(Z - np.max(Z)) 
            a = expZ / np.sum(expZ)
            cost += -(np.sum(y_train[i] * np.log(a)))
            
            dz = a - y_train[i]
            dW = np.dot(dz.reshape(5,1), avg.reshape(1, 50))  # Remove +=, use =
            db = dz 
            
            W = W - learning_rate * dW
            b = b - learning_rate * db
        if(epoch % 10 == 0):
            print(f"epoch {epoch} : cost {cost}")
    return W, b

In [299]:
W, b = model(X_train,Y_oh)

epoch 10 : cost 129.2924046693849
epoch 20 : cost 101.70863817739952
epoch 30 : cost 87.41245576614544
epoch 40 : cost 78.08913407967702
epoch 50 : cost 71.298680783791
epoch 60 : cost 66.03268798332354
epoch 70 : cost 61.783342438152744
epoch 80 : cost 58.25917355598309
epoch 90 : cost 55.276399349740096
epoch 100 : cost 52.71095396252136
epoch 110 : cost 50.47496095377808
epoch 120 : cost 48.50405378250676
epoch 130 : cost 46.74990961988325
epoch 140 : cost 45.175525994086094
epoch 150 : cost 43.75208160388131
epoch 160 : cost 42.45678235337999
epoch 170 : cost 41.271350111727365
epoch 180 : cost 40.18094321818775
epoch 190 : cost 39.17337287174366
epoch 200 : cost 38.23852563501444
epoch 210 : cost 37.36793169123422
epoch 220 : cost 36.55443766959714
epoch 230 : cost 35.79195554614111
epoch 240 : cost 35.075267637048256
epoch 250 : cost 34.399873480122395
epoch 260 : cost 33.761868375397384
epoch 270 : cost 33.157846125630655
epoch 280 : cost 32.58482047185933
epoch 290 : cost 32.04

In [300]:
def predict(sentence , W , b):
    avg = sentence_to_emb(sentence)
    Z = np.dot(W,avg) + b
    expZ = np.exp(Z - np.max(Z)) 
    a = expZ / np.sum(expZ)
    a = np.argmax(a)
    return a

In [301]:
def evaluate(X_test , y_test , W , b ):
    accurate = 0
    for i in range(len(X_test)):
        y_pred = predict(X_test[i], W , b)
        
        if(y_test[i][y_pred] == 1):
            accurate+=1
    return f"{(accurate/len(X_test))*100}% accuracy on the test set"

In [302]:
evaluate(X_test , Y_oh_test , W , b)

'89.0909090909091% accuracy on the test set'

In [313]:
label_to_emoji[predict("I love death",W,b)]

'😔'